In [12]:
import pandas as pd
from scipy.io import loadmat
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from utilities import remove_empty_tweets


C:\Users\maryam\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


100%|██████████████████████████████████████████████████████████████████████████████| 241M/241M [03:21<00:00, 1.30MiB/s]

[2021-02-21 14:24:00,503 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [13]:
train_data_path = 'cleaned_data/cleaned_train_data_for_subtask1.csv'
test_data_path = 'cleaned_data/cleaned_test_data_for_subtask1.csv'
#read files.
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

print("Train set:"% train_data.columns, train_data.shape, len(train_data)) 
print("Test set:"% test_data.columns, test_data.shape, len(test_data)) 



Train set: (20974, 9) 20974
Test set: (4997, 9) 4997


In [14]:
train_data = remove_empty_tweets(train_data, "#2_tweet_clean_V0")
test = remove_empty_tweets(test_data, "#2_tweet_clean_V0")

train_data.head()

,#1_tweetid,#2_tweet,#3_country_label,#2_tweet_clean_V0,#2_tweet_clean_farasaV0,#2_tweet_clean_V1,#2_tweet_clean_V2,#2_tweet_clean_V3,#classes_id
0,TRAIN_0,حاجة حلوة اكيد,Egypt,حاجة حلوة اكيد,حاج ة حلو ة أكيد,حاجه حلوه اكيد,حاجه حلوه اكيد,حاجه حلوه اكيد,0
1,TRAIN_1,عم بشتغلوا للشعب الاميركي اما نحن يكذبوا ويغشو...,Iraq,عم بشتغلوا للشعب الاميركي اما نحن يكذبوا ويغشو...,عم بشتغلوا ل ال شعب ال أميركي اما نحن يكذب وا ...,عم بشتغلوا لشعب الاميركي اما نحن يكذبوا ويغشوا...,عم بشتغلوا لشعب الاميركي يكذبوا ويغشوا ويسرقوا...,عم بشتغلوا لشعب الاميركي يكذبوا ويغشوا ويسرقوا...,1
2,TRAIN_2,ابشر طال عمرك,Saudi_Arabia,ابشر طال عمرك,ابشر طال عمر ك,ابشر طال عمرك,ابشر طال عمرك,ابشر طال عمرك,2
3,TRAIN_3,منطق 2017: أنا والغريب علي إبن عمي وأنا والغري...,Mauritania,منطق أنا والغريب علي إبن عمي وأنا والغريب وإب...,منطق أنا و ال غريب علي إبن عمي و أنا و ال غريب...,منطق انا والغريب علي ابن عمي وانا والغريب وابن...,منطق والغريب ابن عمي وانا والغريب وابن عمي اخو...,منطق والغريب ابن عمي وانا وابن اخويا قطع العلا...,3
4,TRAIN_4,شهرين وتروح والباقي غير صيف ملينا,Algeria,شهرين وتروح والباقي غير صيف ملينا,شهر ين و تروح و ال باقي غير صيف ملي نا,شهرين وتروح والباقي غير صيف ملينا,شهرين وتروح والباقي صيف ملينا,شهرين وتروح والباقي صيف ملينا,4


In [15]:
#prepare train and test data.
X_train = train_data['#2_tweet_clean_V0'].tolist()
y_train = train_data['#classes_id'].tolist()
X_test = test_data['#2_tweet_clean_V0'].tolist()
y_test = test_data['#classes_id'].tolist()


In [16]:
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(X_train,y_train,test_size=0.3, random_state=42)

print(len(x_train),len(x_valid))

14681 6293


In [18]:
#Create vocabulary

Tfidf_vect = TfidfVectorizer(max_features=5000, ngram_range=(1,3))
Tfidf_vect.fit(X_train)
Train_X_Tfidf = Tfidf_vect.transform(x_train)
Test_X_Tfidf = Tfidf_vect.transform(x_valid)

print(Tfidf_vect.vocabulary_)

{'حاجة': 1996, 'حلوة': 2118, 'اكيد': 413, 'عم': 2958, 'اما': 1166, 'نحن': 4165, 'طال': 2771, 'عمرك': 2963, 'طال عمرك': 2772, 'أنا': 86, 'علي': 2941, 'عمي': 2979, 'وأنا': 4383, 'قطع': 3272, 'العلاقات': 843, 'مع': 3903, 'قطر': 3271, 'موريتانيا': 4101, 'شهرين': 2655, 'والباقي': 4423, 'غير': 3054, 'صيف': 2755, 'ملينا': 3992, 'والله': 4434, 'ما': 3653, 'حد': 2045, 'ولا': 4594, 'منك': 4066, 'انتى': 1244, 'اساسا': 290, 'والله ما': 4442, 'ما حد': 3661, 'نفس': 4205, 'الوقت': 1145, 'على': 2924, 'ان': 1186, 'اي': 1313, 'هدف': 4287, 'دايما': 2235, 'عشان': 2883, 'نفسنا': 4210, 'الفاضي': 874, 'نفس الوقت': 4207, 'خرا': 2170, 'بقا': 1563, 'علشان': 2919, 'عليكي': 2951, 'نفسك': 4209, 'دخل': 2237, 'هو': 4352, 'حلو': 2115, 'بس': 1489, 'ليك': 3639, 'عدنا': 2861, 'حلو بس': 2117, 'أي': 107, 'بصراحة': 1520, 'مسلسل': 3859, 'هيك': 4376, 'بتذكر': 1422, 'كان': 3334, 'اسمها': 309, 'في': 3154, 'أحمد': 24, 'ربي': 2374, 'كي': 3473, 'يجي': 4790, 'واحد': 4394, 'منهم': 4072, 'واحد منهم': 4398, 'يسطا': 4859, 'لازم': 3516

In [19]:
test_X_Tfidf = Tfidf_vect.transform(X_test)

In [20]:
print(test_X_Tfidf)


  (0, 4675)	0.27305724845033935
  (0, 3511)	0.6392577625266642
  (0, 3495)	0.6028721132512005
  (0, 2941)	0.39157945194068133
  (1, 4335)	1.0
  (2, 4823)	0.47702343228537536
  (2, 4675)	0.22632826746090845
  (2, 4612)	0.5349172822777034
  (2, 2371)	0.3594090315330284
  (2, 1020)	0.5530938524083864
  (3, 4394)	0.3451176473848934
  (3, 3603)	0.402533813974211
  (3, 3200)	0.544354745518127
  (3, 2996)	0.4498042750026385
  (3, 2673)	0.3241854091800918
  (3, 1313)	0.33929070686783325
  (4, 4369)	0.5993305425311632
  (4, 2606)	0.6833976333917657
  (4, 2034)	0.41685797996900037
  (5, 4113)	0.6205005441627012
  (5, 2686)	0.6268777541352989
  (5, 1336)	0.38337226790744816
  (5, 1194)	0.27391433015374644
  (6, 4121)	0.6033244102866313
  (6, 3495)	0.40774215436993005
  :	:
  (4990, 1373)	0.57639400283882
  (4991, 4675)	0.3332688253929455
  (4991, 2457)	0.7957620304112571
  (4991, 1489)	0.3671424668921525
  (4991, 1194)	0.3477083403965235
  (4992, 3851)	0.6135489960351875
  (4992, 3154)	0.29563477

#  Radial Basis Function  (RBF) kernel

In [21]:
import timeit

start = timeit.default_timer()

SVM = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma=2)
SVM.fit(Train_X_Tfidf,y_train)

stop = timeit.default_timer()
print('Train time: ', stop - start)  

# predict the labels on validation dataset
predictions_SVM_valid = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM_valid, y_valid)*100)



Train time:  33.2202941999999
SVM Accuracy Score ->  31.82901636739234


In [22]:

predictions_SVM = SVM.predict(test_X_Tfidf)

print(classification_report(predictions_SVM, y_test))

              precision    recall  f1-score   support

           0       0.92      0.32      0.48      2965
           1       0.53      0.30      0.39      1150
           2       0.25      0.28      0.27       462
           3       0.00      0.00      0.00         0
           4       0.25      0.46      0.33       236
           5       0.03      0.21      0.05        33
           6       0.03      0.14      0.05        85
           7       0.02      0.60      0.03         5
           8       0.01      0.50      0.01         2
           9       0.01      0.43      0.03         7
          10       0.00      0.00      0.00         1
          11       0.01      1.00      0.01         1
          12       0.01      0.33      0.02         3
          13       0.05      0.44      0.10        39
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0
          16       0.01      0.25      0.02         4
          17       0.00    

C:\Users\maryam\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Quadratic (poly) kernel

In [23]:
startquad = timeit.default_timer()

SVMquad = svm.SVC(C=1.0, kernel='poly', degree=3, gamma=2)
SVMquad.fit(Train_X_Tfidf,y_train)

stopquad = timeit.default_timer()
print('Train time: ', stopquad - startquad)  

# predict the labels on validation dataset
predictions_SVMquad = SVMquad.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVMquad, y_valid)*100)



Train time:  33.4499998
SVM Accuracy Score ->  26.108374384236456


In [24]:

predictions_SVMquad_test = SVMquad.predict(test_X_Tfidf)

print(classification_report(predictions_SVMquad_test, y_test))

              precision    recall  f1-score   support

           0       0.87      0.26      0.41      3438
           1       0.36      0.33      0.35       720
           2       0.11      0.22      0.15       261
           3       0.02      0.33      0.04         3
           4       0.15      0.37      0.21       174
           5       0.03      0.11      0.04        65
           6       0.04      0.12      0.06       110
           7       0.02      0.15      0.03        20
           8       0.01      0.25      0.02         8
           9       0.03      0.16      0.05        38
          10       0.00      0.00      0.00         1
          11       0.01      0.11      0.02        19
          12       0.03      0.25      0.05        12
          13       0.06      0.23      0.09        80
          14       0.00      0.00      0.00         3
          15       0.00      0.00      0.00         3
          16       0.01      0.07      0.02        15
          17       0.00    

In [25]:
print(classification_report(predictions_SVMquad, y_valid))

              precision    recall  f1-score   support

           0       0.87      0.25      0.38      4433
           1       0.37      0.35      0.36       877
           2       0.10      0.21      0.13       293
           3       0.01      0.50      0.03         2
           4       0.17      0.43      0.24       218
           5       0.02      0.12      0.03        68
           6       0.05      0.18      0.08       120
           7       0.05      0.39      0.09        33
           8       0.02      0.21      0.04        19
           9       0.03      0.29      0.05        24
          10       0.03      0.29      0.06         7
          11       0.00      0.00      0.00        19
          12       0.02      0.10      0.03        20
          13       0.06      0.23      0.10       105
          14       0.00      0.00      0.00         4
          15       0.00      0.00      0.00         4
          16       0.00      0.00      0.00        13
          17       0.02    

# Linear kernel

In [26]:
startlin = timeit.default_timer()

SVMlin = svm.SVC(C=1.0, kernel='linear', degree=3, gamma=2)
SVMlin.fit(Train_X_Tfidf,y_train)

stoplin = timeit.default_timer()
print('Train time: ', stoplin - startlin)  

# predict the labels on validation dataset
predictions_SVMlin = SVMlin.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVMlin, y_valid)*100)



Train time:  19.426574300000084
SVM Accuracy Score ->  37.2000635626887


In [27]:

predictions_SVMlin_test = SVMlin.predict(test_X_Tfidf)

print(classification_report(predictions_SVMlin_test, y_test))

              precision    recall  f1-score   support

           0       0.84      0.43      0.57      2038
           1       0.50      0.32      0.39      1026
           2       0.38      0.25      0.30       800
           3       0.13      0.88      0.23         8
           4       0.41      0.40      0.41       433
           5       0.09      0.17      0.12       144
           6       0.11      0.15      0.13       269
           7       0.06      0.33      0.10        30
           8       0.03      0.21      0.05        19
           9       0.04      0.45      0.08        20
          10       0.00      0.00      0.00         4
          11       0.02      0.11      0.03        27
          12       0.03      1.00      0.06         3
          13       0.17      0.35      0.23       153
          14       0.00      0.00      0.00         5
          15       0.00      0.00      0.00         1
          16       0.01      0.20      0.02         5
          17       0.02    

In [28]:
print(classification_report(predictions_SVMlin, y_valid))

              precision    recall  f1-score   support

           0       0.84      0.41      0.55      2559
           1       0.51      0.34      0.41      1248
           2       0.40      0.26      0.32       963
           3       0.08      0.86      0.14         7
           4       0.45      0.44      0.45       566
           5       0.15      0.34      0.21       179
           6       0.18      0.27      0.21       305
           7       0.18      0.62      0.28        77
           8       0.12      0.65      0.20        37
           9       0.07      0.42      0.11        40
          10       0.31      0.86      0.45        22
          11       0.04      0.23      0.06        30
          12       0.01      0.33      0.02         3
          13       0.23      0.37      0.28       228
          14       0.06      0.44      0.10         9
          15       0.00      0.00      0.00         1
          16       0.01      0.50      0.01         2
          17       0.02    

# Sigmoid kernel

In [29]:
startsig = timeit.default_timer()

SVMsig = svm.SVC(C=1.0, kernel='sigmoid', degree=3, gamma=2)
SVMsig.fit(Train_X_Tfidf,y_train)

stopsig = timeit.default_timer()
print('Train time: ', stopsig - startsig)  

# predict the labels on validation dataset
predictions_SVMsig = SVMsig.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVMsig, y_valid)*100)



Train time:  16.030429400000003
SVM Accuracy Score ->  36.13538852693469


In [30]:
print(classification_report(predictions_SVMsig, y_valid))

              precision    recall  f1-score   support

           0       0.81      0.42      0.55      2433
           1       0.49      0.32      0.39      1246
           2       0.37      0.25      0.30       935
           3       0.09      0.88      0.17         8
           4       0.45      0.44      0.45       571
           5       0.16      0.30      0.20       208
           6       0.17      0.24      0.20       323
           7       0.20      0.62      0.30        86
           8       0.10      0.32      0.16        65
           9       0.06      0.25      0.10        64
          10       0.32      0.80      0.46        25
          11       0.05      0.19      0.07        47
          12       0.01      0.17      0.02         6
          13       0.23      0.38      0.29       226
          14       0.03      0.25      0.05         8
          15       0.00      0.00      0.00         1
          16       0.02      0.30      0.04        10
          17       0.02    

In [31]:

predictions_SVMsig_test = SVMsig.predict(test_X_Tfidf)

print(classification_report(predictions_SVMsig_test, y_test))

              precision    recall  f1-score   support

           0       0.82      0.43      0.56      1981
           1       0.49      0.32      0.38      1022
           2       0.37      0.25      0.30       753
           3       0.09      1.00      0.17         5
           4       0.36      0.38      0.37       408
           5       0.09      0.14      0.11       166
           6       0.12      0.15      0.13       279
           7       0.07      0.30      0.11        40
           8       0.04      0.16      0.06        37
           9       0.05      0.29      0.08        34
          10       0.00      0.00      0.00         6
          11       0.04      0.14      0.06        43
          12       0.02      0.29      0.04         7
          13       0.18      0.33      0.23       169
          14       0.00      0.00      0.00         4
          15       0.00      0.00      0.00         1
          16       0.01      0.11      0.02         9
          17       0.02    